In [1]:
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as plt
import os.path
import csv

In [2]:
dataset_path = '/Users/Johannes/Uni/Master/Master Arbeit/datasets/ml-20m/'
ratings = os.path.join(dataset_path, 'ratings-sample-4.4M.csv')
coo_path = os.path.join(dataset_path, '_coordinate matrix')
sr_path = os.path.join(dataset_path, '_row matrix')
block_path = os.path.join(dataset_path, '_block matrix')

In [57]:
# Coordinate Matrix
with open(ratings, 'rb') as ratings_file:
    reader = csv.DictReader(ratings_file, delimiter=',')
    for row in reader:
        with open(os.path.join(coo_path, row['userId'] + "_" + row['movieId']), 'w') as coo_file:
            coo_file.write(row['rating'])

KeyboardInterrupt: 

In [58]:
def write_user_ratings_to_file(user_id, data):
    user_ratings_path = os.path.join(sr_path, user_id + ".csv")
    with open(user_ratings_path, 'wb') as ratings_file:
        writer = csv.writer(ratings_file)
        writer.writerow(['movieId','rating'])
        writer.writerows(data)
test_data = [[1,2],[1,2],[1,2]]
write_user_ratings_to_file('1', test_data)

In [59]:
# Row Matrix
with open(ratings, 'rb') as ratings_file:
    reader = csv.DictReader(ratings_file, delimiter=',')
    current_user_id = "1"
    user_ratings = []    
    for rating in reader:
        if rating['userId'] != current_user_id:
            write_user_ratings_to_file(current_user_id, user_ratings)
            current_user_id = rating['userId']
            user_ratings = [[rating['movieId'], rating['rating']]]
        else:
            user_ratings.append([rating['movieId'], rating['rating']])
    write_user_ratings_to_file(current_user_id, user_ratings)


### tester for s3 download with pagination

In [48]:
import boto3

bucket = 'jmue-s3-benchmark'
prefix = 'coordinate_matrix/1_'

client = boto3.client('s3', region_name='eu-central-1')
paginator = client.get_paginator('list_objects')
page_iterator = paginator.paginate(Bucket=bucket,
                                   Prefix=prefix)

In [54]:
i = 0
pages = []
for page in page_iterator:
    pages.append(page['Contents'])
    i += 1
    if i > 15:
        break

In [55]:
keys = []
for page in pages:
    for item in page:
        keys.append(item['Key'])
keys

[u'coordinate_matrix/1_1009',
 u'coordinate_matrix/1_1036',
 u'coordinate_matrix/1_1079',
 u'coordinate_matrix/1_1080',
 u'coordinate_matrix/1_1089',
 u'coordinate_matrix/1_1090',
 u'coordinate_matrix/1_1097',
 u'coordinate_matrix/1_112',
 u'coordinate_matrix/1_1136',
 u'coordinate_matrix/1_1193',
 u'coordinate_matrix/1_1196',
 u'coordinate_matrix/1_1198',
 u'coordinate_matrix/1_1200',
 u'coordinate_matrix/1_1201',
 u'coordinate_matrix/1_1208',
 u'coordinate_matrix/1_1214',
 u'coordinate_matrix/1_1215',
 u'coordinate_matrix/1_1217',
 u'coordinate_matrix/1_1219',
 u'coordinate_matrix/1_1222',
 u'coordinate_matrix/1_1240',
 u'coordinate_matrix/1_1243',
 u'coordinate_matrix/1_1246',
 u'coordinate_matrix/1_1249',
 u'coordinate_matrix/1_1258',
 u'coordinate_matrix/1_1259',
 u'coordinate_matrix/1_1261',
 u'coordinate_matrix/1_1262',
 u'coordinate_matrix/1_1266',
 u'coordinate_matrix/1_1278',
 u'coordinate_matrix/1_1291',
 u'coordinate_matrix/1_1304',
 u'coordinate_matrix/1_1321',
 u'coordina

### this is the actual code

In [1]:
import boto3

In [15]:
bucket = 'jmue-s3-benchmark'
prefix = 'coordinate_matrix/1_'

s3_client = boto3.client('s3', region_name='eu-central-1')
paginator = s3_client.get_paginator('list_objects')
page_iterator = paginator.paginate(Bucket=bucket,
                                   Prefix=prefix)

In [3]:
keys = []
for page in page_iterator:
    for item in page['Contents']:
        keys.append(item['Key'])
keys

[u'coordinate_matrix/1_1009',
 u'coordinate_matrix/1_1036',
 u'coordinate_matrix/1_1079',
 u'coordinate_matrix/1_1080',
 u'coordinate_matrix/1_1089',
 u'coordinate_matrix/1_1090',
 u'coordinate_matrix/1_1097',
 u'coordinate_matrix/1_112',
 u'coordinate_matrix/1_1136',
 u'coordinate_matrix/1_1193',
 u'coordinate_matrix/1_1196',
 u'coordinate_matrix/1_1198',
 u'coordinate_matrix/1_1200',
 u'coordinate_matrix/1_1201',
 u'coordinate_matrix/1_1208',
 u'coordinate_matrix/1_1214',
 u'coordinate_matrix/1_1215',
 u'coordinate_matrix/1_1217',
 u'coordinate_matrix/1_1219',
 u'coordinate_matrix/1_1222',
 u'coordinate_matrix/1_1240',
 u'coordinate_matrix/1_1243',
 u'coordinate_matrix/1_1246',
 u'coordinate_matrix/1_1249',
 u'coordinate_matrix/1_1258',
 u'coordinate_matrix/1_1259',
 u'coordinate_matrix/1_1261',
 u'coordinate_matrix/1_1262',
 u'coordinate_matrix/1_1266',
 u'coordinate_matrix/1_1278',
 u'coordinate_matrix/1_1291',
 u'coordinate_matrix/1_1304',
 u'coordinate_matrix/1_1321',
 u'coordina

In [12]:
def download_user_ratings(user_id):
    '''Download from coordinate matrix'''
    bucket = 'jmue-s3-benchmark'
    prefix = 'coordinate_matrix/' + str(user_id) + '_'

    s3_client = boto3.client('s3', region_name='eu-central-1')
    paginator = s3_client.get_paginator('list_objects')
    page_iterator = paginator.paginate(Bucket=bucket,
                                       Prefix=prefix)
    keys = []
    for page in page_iterator:
        for item in page['Contents']:
            keys.append(item['Key'])
            
    # TODO: make sure the directory exists! /tmp/coordinate_matrix...
    for key in keys:
        s3_client.download_file('jmue-s3-benchmark', key, '/tmp/' + str(key) + '.csv')

In [5]:
download_user_ratings(1)

In [31]:
def download_row(user_id):
    bucket = 'jmue-s3-benchmark'
    key = 'row_matrix/' + str(user_id) + '.csv'
    s3_client.download_file(bucket, key, '/tmp/' + str(key))

In [34]:
download_row(user_id=5)

In [9]:
# TODO: implement
def download_rows(number_of_rows):
    '''Download from coordinate matrix'''
    bucket = 'jmue-s3-benchmark'
    prefix = 'row_matrix/'

    paginator = s3_client.get_paginator('list_objects')
    page_iterator = paginator.paginate(Bucket=bucket, Prefix=prefix)
    
    keys = []
    for page in page_iterator:
        for item in page['Contents']:
            keys.append(item['Key'])
    for key in keys:
        s3_client.download_file(bucket, key, '/tmp/' + str(key) + '.csv')
    